# 初期設定

## ライブラリ

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import seaborn as sns

# チューニンング方法

## グリッドサーチ/ランダムサーチ
- ランダムサーチの方が効率が良い by Bergstra and Bengio  

 ![Grid or Random](/Users/nozawayuta/GitHub/kagglebook/misc/gridrandom.png "Grid or Random")

In [3]:
#パラメータ1,パラメータ2の候補
param1_list = [3, 5, 7, 9]
param2_list = [1, 2, 3, 4, 5]

# グリッドサーチで探索するパラメータの組み合わせ
grid_search_params = []
for p1 in param1_list:
    for p2 in param2_list:
        grid_search_params.append((p1, p2))

# ランダムサーチで探索するパラメータの組み合わせ
random_search_params = []
trials = 15
for i in range(trials):
    p1 = np.random.choice(param1_list)
    p2 = np.random.choice(param2_list)
    random_search_params.append((p1, p2))

## ベイズ最適化 Bayesian Optimization  
- 過去の探索から誤差を読み取り精度の高いパラメータに近づけていく考え方  
- hyperoptライブラリが比較的よく使われる 
- その他ライブラリ   
    - optuna
    - gpyopt  
    - spearmint  
    - sckit-optimize
- パラメタ調整フロー  
    1. ベースラインとなるパラメータで学習
    1. 1-3種類, 2-5候補で**グリッドサーチ**を行う
    1. 本格的にパラメータチューニングを行う場合には**ベイズ最適化**を使う


### hyperopt
- TPE(Tree-structured Parzen Estimator)アルゴリズムを使用
- 設定内容
    - 最小化したい評価指標の指定
        - accuracyなど高い方が良い評価指標の際には正負を反転させる必要有り
    - 探索するパラメータの範囲の定義
        - 事前分布の定義
            - 一様分布（決定木の深さ）
     - 探索回数の指定
        - 目安は100回程度
- 撮り得るパラメータの組み合わせの集合を**パラメータ空間**という

In [ ]:
class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred


from hyperopt import hp

space = {
    'activation': hp.choice('activation', ['prelu', 'relu']),
    'dropout': hp.uniform('dropout', 0, 0.2),
    'units': hp.quniform('units', 32, 256, 32),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.00001), np.log(0.01)),
}

# -----------------------------------
# hyperoptを使ったパラメータ探索
# -----------------------------------
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss

# 最小化したい評価指標のスコアを返す関数を作成する
## validationに対して推論して評価指標のスコアを計算する処理を行う
def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# 探索するパラメータの空間を指定する
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

# hyperoptによるパラメータ探索の実行
max_evals = 10
trials = Trials()
history = []
# 作成した関数、パラメータ空間、アルゴリズム、Trials(), 探索回数(max_evals)をしてすることで探索開始
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

### optuna
- アルゴリズムはTPE（hyperoptと同様）
- APIが使いやすくなっている→効率的なチューニングができる
    - Define-by-Run
        - パラメータ空間を別途定義するhyperoptと異なり、モデル記述の中で定義、決定する
    - 学習曲線を用いた試行の枝刈り
    - 並列分散最適化
    

## GBDTパラメータとチューニング
 ![param](/Users/nozawayuta/GitHub/kagglebook/misc/xgboost_param.png "param")
